In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.neighbors import NearestNeighbors
import numpy as np
import time

In [2]:
game = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv')
print(game.shape)
game.Critic_Score=game.Critic_Score.fillna(0)
game.Critic_Count=game.Critic_Count.fillna(0)
game.User_Score=game.User_Score.fillna(0)
game.User_Count=game.User_Count.fillna(0)
game.Developer=game.Developer.fillna('nan')
game.Rating=game.Rating.fillna('nan')
game.head()

(16719, 16)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.0,0.0,0,0.0,nan,nan
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,0.0,0.0,0,0.0,nan,nan


In [3]:
game.Name=game.Name.astype('str')
game.Platform=game.Platform.astype('str')
game.Genre=game.Genre.astype('str')
game.Publisher=game.Publisher.astype('str')
game.Developer=game.Developer.astype('str')
game.Rating=game.Rating.astype('str')
game.User_Score.loc[game.User_Score == 'tbd'] = 0
game.User_Score=game.User_Score.astype('float')

list_genres = np.unique(game.Genre).tolist()
list_platforms = np.unique(game.Platform).tolist()
list_ratings = np.unique(game.Rating).tolist()

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


![alt text](CodeCogsEqn.png)

In [4]:
gaussian_filter_2d = lambda x,y,sigma: np.exp((-x**2 -y**2)/(2 * sigma ** 2))

In [7]:
def dynamic_dimension(chosen_index): 
    labels = []
    if game.Publisher.iloc[chosen_index]:
        labels += [game.Publisher.iloc[chosen_index]]
    if game.Developer.iloc[chosen_index]:
        labels += [game.Developer.iloc[chosen_index]]
    return labels

In [20]:
def add_dimension(df, variables, columns):
    shape_len = df.shape[1]
    df = np.hstack([df, np.zeros((df.shape[0], len(variables)))])
    for i in columns:
        for k in range(df.shape[0]):
            if not df[k,i]:
                continue
            try:
                if df[k,i] in variables:
                    df[k,shape_len+variables.index(df[k,i])] += 1
            except Exception as e:
                print(e)
                continue
    return df

In [27]:
def recommend(chosen_index, count_neigh):
    df_copy = game.copy().values
    variables = list_genres + list_platforms + list_ratings + dynamic_dimension(chosen_index)
    df_copy = add_dimension(df_copy, variables, [4,14])
    df_copy = df_copy[:,game.shape[1]:].astype('float')
    df_copy = df_copy[~np.isnan(df_copy).any(axis=1)]
    neighbors = NearestNeighbors(n_neighbors=count_neigh+1, algorithm='auto', metric='sqeuclidean').fit(df_copy)
    xtest = df_copy[chosen_index, :]
    xtest = xtest.reshape(1, -1)
    distances, indices = neighbors.kneighbors(xtest)
    results = indices[0][:].tolist()
    try:
        del results[results.index(chosen_index)]
    except:
        results=results[:count_neigh]
        pass
    return results

In [31]:
battlefield_index = np.argmax([fuzz.ratio('battlefield', i) for i in game.Name.values])
suggest_number = 10
print('battlefield selected:')
print(game.iloc[battlefield_index,:])
recommended = recommend(battlefield_index, suggest_number)
game.iloc[recommended,:]

battlefield selected:
Name                 Battlefield 3
Platform                      X360
Year_of_Release               2011
Genre                      Shooter
Publisher          Electronic Arts
NA_Sales                      4.46
EU_Sales                      2.11
JP_Sales                      0.06
Other_Sales                   0.69
Global_Sales                  7.32
Critic_Score                    84
Critic_Count                    57
User_Score                     7.4
User_Count                    2142
Developer                  EA DICE
Rating                           M
Name: 101, dtype: object


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
14919,Battlefield 2142,PC,2006.0,Shooter,Electronic Arts,0.00,0.02,0.00,0.00,0.03,80.0,48.0,6.8,326.0,EA DICE,T
357,Battlefield 4,PS4,2013.0,Shooter,Electronic Arts,1.35,1.55,0.17,0.51,3.59,85.0,18.0,6.9,1580.0,EA DICE,M
8492,Mirror's Edge Catalyst,XOne,2016.0,Platform,Electronic Arts,0.09,0.06,0.00,0.01,0.16,72.0,46.0,4.4,266.0,EA DICE,T
2859,Battlefield: Bad Company 2,PC,2010.0,Shooter,Electronic Arts,0.19,0.53,0.00,0.00,0.72,87.0,46.0,8.3,1467.0,EA DICE,M
1810,Mirror's Edge,PS3,2008.0,Platform,Electronic Arts,0.31,0.58,0.02,0.22,1.13,79.0,66.0,7.8,362.0,EA DICE,T
1437,Battlefield 4,PC,2013.0,Shooter,Electronic Arts,0.39,0.87,0.00,0.10,1.35,81.0,52.0,6.0,3517.0,EA DICE,M
5320,Mirror's Edge Catalyst,PS4,2016.0,Platform,Electronic Arts,0.11,0.16,0.02,0.05,0.35,69.0,50.0,4.7,578.0,EA DICE,T
847,Battlefield 4,XOne,2013.0,Shooter,Electronic Arts,1.25,0.58,0.00,0.19,2.02,81.0,5.0,6.6,480.0,EA DICE,M
485,Battlefield: Bad Company 2,PS3,2010.0,Shooter,Electronic Arts,1.33,1.10,0.08,0.44,2.95,88.0,59.0,8.5,572.0,EA DICE,M
1311,Battlefield: Bad Company,X360,2008.0,Shooter,Electronic Arts,0.81,0.46,0.04,0.15,1.46,83.0,70.0,7.7,263.0,EA DICE,T


In [41]:
def sequel(title_1, title_2):    
    if fuzz.ratio(title_1, title_2) > 70 or fuzz.token_set_ratio(title_1, title_2) > 70:
        return True
    else:
        return False

def sort_distribution(title_main, std_user_count, std_critic_count, std_date, 
                      title, user_score, user_count, critic_score, critic_count, 
                      date, global_sale):
    if user_score:
        factor_e_left = gaussian_filter_2d(user_score, user_count, std_user_count)
    else:
        factor_e_left = -999999999
    if sequel(title_main, title):
        gaussian_left = 0
    else:
        gaussian_left = (1 / (np.pi * 2 * std_user_count ** 2)) * factor_e_left
        
    if critic_score:
        factor_e_middle = gaussian_filter_2d(critic_score, critic_count, std_critic_count)
    else:
        factor_e_middle = -999999999
    gaussian_middle = (1 / (np.pi * 2 * std_critic_count ** 2)) * factor_e_middle
    
    if global_sale:
        factor_e_right = gaussian_filter_2d(global_sale, date, std_date)
    else:
        factor_e_right = -999999999
        
    gaussian_right = (1 / (np.pi * 2 * std_date ** 2)) * factor_e_right
    return (gaussian_left + gaussian_middle + gaussian_right) / 3

In [44]:
def extract_ratings(chosen_index, list_films, count_neigh):
    parametres_game = ['_' for _ in range(count_neigh)]
    users, critics, dates = [], [], []
    for i, index in enumerate(list_films):
        parametres_game[i] = game[['Name','User_Score','User_Count','Critic_Score',
                                   'Critic_Count', 'Year_of_Release', 'Global_Sales']].iloc[index].tolist()
        parametres_game[i].append(index)
        users.append(parametres_game[i][2])
        critics.append(parametres_game[i][4])
        dates.append(parametres_game[i][5])
    
    title_main = game['Name'].iloc[chosen_index]
    users.append(game['User_Count'].iloc[chosen_index])
    critics.append(game['Critic_Count'].iloc[chosen_index])
    dates.append(game['Year_of_Release'].iloc[chosen_index])
    std_users = np.std(users)
    std_critics = np.std(critics)
    std_dates = np.std(dates)
    results_gaussian = [sort_distribution(title_main,std_users,std_critics, std_dates,
                                          x[0],x[1],x[2],x[3],x[4],x[5],x[6]) for x in parametres_game]
    results = np.array(parametres_game)[(np.array(results_gaussian).argsort())]
    return results[:,-1].tolist()

In [50]:
battlefield_index = np.argmax([fuzz.ratio('battlefield', i) for i in game.Name.values])
suggest_number = 10
print('battlefield selected:')
print(game.iloc[battlefield_index,:])
recommended = recommend(battlefield_index, suggest_number)
sorted_recommended = np.array(extract_ratings(battlefield_index, recommended, suggest_number)).astype('int')
game.iloc[sorted_recommended,:]

battlefield selected:
Name                 Battlefield 3
Platform                      X360
Year_of_Release               2011
Genre                      Shooter
Publisher          Electronic Arts
NA_Sales                      4.46
EU_Sales                      2.11
JP_Sales                      0.06
Other_Sales                   0.69
Global_Sales                  7.32
Critic_Score                    84
Critic_Count                    57
User_Score                     7.4
User_Count                    2142
Developer                  EA DICE
Rating                           M
Name: 101, dtype: object


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1311,Battlefield: Bad Company,X360,2008.0,Shooter,Electronic Arts,0.81,0.46,0.04,0.15,1.46,83.0,70.0,7.7,263.0,EA DICE,T
485,Battlefield: Bad Company 2,PS3,2010.0,Shooter,Electronic Arts,1.33,1.10,0.08,0.44,2.95,88.0,59.0,8.5,572.0,EA DICE,M
2859,Battlefield: Bad Company 2,PC,2010.0,Shooter,Electronic Arts,0.19,0.53,0.00,0.00,0.72,87.0,46.0,8.3,1467.0,EA DICE,M
1437,Battlefield 4,PC,2013.0,Shooter,Electronic Arts,0.39,0.87,0.00,0.10,1.35,81.0,52.0,6.0,3517.0,EA DICE,M
14919,Battlefield 2142,PC,2006.0,Shooter,Electronic Arts,0.00,0.02,0.00,0.00,0.03,80.0,48.0,6.8,326.0,EA DICE,T
357,Battlefield 4,PS4,2013.0,Shooter,Electronic Arts,1.35,1.55,0.17,0.51,3.59,85.0,18.0,6.9,1580.0,EA DICE,M
847,Battlefield 4,XOne,2013.0,Shooter,Electronic Arts,1.25,0.58,0.00,0.19,2.02,81.0,5.0,6.6,480.0,EA DICE,M
5320,Mirror's Edge Catalyst,PS4,2016.0,Platform,Electronic Arts,0.11,0.16,0.02,0.05,0.35,69.0,50.0,4.7,578.0,EA DICE,T
1810,Mirror's Edge,PS3,2008.0,Platform,Electronic Arts,0.31,0.58,0.02,0.22,1.13,79.0,66.0,7.8,362.0,EA DICE,T
8492,Mirror's Edge Catalyst,XOne,2016.0,Platform,Electronic Arts,0.09,0.06,0.00,0.01,0.16,72.0,46.0,4.4,266.0,EA DICE,T


In [51]:
outputs, i = [], 0
while len(outputs) < suggest_number:
    recommended = recommend(battlefield_index, suggest_number * (i+2))
    if not sequel(game['Name'].iloc[battlefield_index], game['Name'].iloc[recommended[i]]) and recommended[i] not in outputs:
        outputs.append(recommended[i])
    i += 1
sorted_recommended = np.array(extract_ratings(battlefield_index,outputs,suggest_number)).astype('int')

In [52]:
game.iloc[sorted_recommended,:]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1810,Mirror's Edge,PS3,2008.0,Platform,Electronic Arts,0.31,0.58,0.02,0.22,1.13,79.0,66.0,7.8,362.0,EA DICE,T
1744,Mirror's Edge,X360,2008.0,Platform,Electronic Arts,0.51,0.51,0.01,0.13,1.16,79.0,82.0,7.6,328.0,EA DICE,T
6627,NHL 08,PS3,2007.0,Action,Electronic Arts,0.16,0.06,0.00,0.03,0.25,86.0,23.0,6.9,23.0,EA Canada,E10+
4668,EA Sports UFC 2,XOne,2016.0,Sports,Electronic Arts,0.24,0.14,0.00,0.04,0.41,76.0,42.0,6.0,77.0,EA Sports,T
8492,Mirror's Edge Catalyst,XOne,2016.0,Platform,Electronic Arts,0.09,0.06,0.00,0.01,0.16,72.0,46.0,4.4,266.0,EA DICE,T
12369,Mirror's Edge Catalyst,PC,2016.0,Platform,Electronic Arts,0.01,0.05,0.00,0.00,0.06,74.0,24.0,4.4,606.0,EA DICE,T
647,Need for Speed Carbon: Own the City,PSP,2006.0,Racing,Electronic Arts,0.87,0.97,0.00,0.58,2.42,73.0,14.0,7.9,39.0,EA Canada,E10+
6579,Madden NFL 2005,DS,2004.0,Sports,Electronic Arts,0.24,0.00,0.00,0.02,0.26,68.0,25.0,7.7,7.0,Exient Entertainment,E
7547,Need for Speed Carbon: Own the City,DS,2006.0,Racing,Electronic Arts,0.17,0.01,0.00,0.02,0.20,70.0,6.0,7.5,15.0,Exient Entertainment,E
6605,NASCAR 09,PS3,2008.0,Racing,Electronic Arts,0.22,0.01,0.00,0.02,0.25,65.0,22.0,7.5,10.0,EA Games,E


In [57]:
def suggest_me_games(keyword, suggest_count, remove_sequel=True):
    lasttime = time.time()
    your_index = np.argmax([fuzz.ratio(keyword, i) for i in game.Name.values])
    print('your selection:')
    print(game.iloc[your_index,:])
    if remove_sequel:
        outputs, i = [], 0
        while len(outputs) < suggest_count:
            recommended = recommend(your_index, suggest_count * (i+2))
            if not sequel(game['Name'].iloc[battlefield_index], game['Name'].iloc[recommended[i]]) and recommended[i] not in outputs:
                outputs.append(recommended[i])
            i += 1
        results = np.array(extract_ratings(your_index,outputs,suggest_count)).astype('int')
    else:
        recommended = recommend(your_index, suggest_count)
        results = np.array(extract_ratings(your_index,recommended,suggest_count)).astype('int')
    print('time taken to suggest:', time.time()-lasttime, 'seconds')
    return results

In [59]:
game.iloc[suggest_me_games('battlefield', 15, remove_sequel=True),:]

your selection:
Name                 Battlefield 3
Platform                      X360
Year_of_Release               2011
Genre                      Shooter
Publisher          Electronic Arts
NA_Sales                      4.46
EU_Sales                      2.11
JP_Sales                      0.06
Other_Sales                   0.69
Global_Sales                  7.32
Critic_Score                    84
Critic_Count                    57
User_Score                     7.4
User_Count                    2142
Developer                  EA DICE
Rating                           M
Name: 101, dtype: object
time taken to suggest: 2.5386440753936768 seconds


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1356,Dead Space 2,PS3,2011.0,Shooter,Electronic Arts,0.73,0.48,0.00,0.21,1.42,89.0,78.0,8.4,535.0,Visceral Games,M
1744,Mirror's Edge,X360,2008.0,Platform,Electronic Arts,0.51,0.51,0.01,0.13,1.16,79.0,82.0,7.6,328.0,EA DICE,T
905,Need for Speed: Shift,X360,2009.0,Racing,Electronic Arts,0.73,0.96,0.01,0.22,1.91,83.0,75.0,6.5,134.0,Slightly Mad Studios,E
2623,Fight Night Champion,X360,2011.0,Fighting,Electronic Arts,0.40,0.31,0.00,0.07,0.78,86.0,59.0,7.4,105.0,EA Canada,M
2400,Fight Night Round 2,PS2,2005.0,Fighting,Electronic Arts,0.72,0.03,0.00,0.12,0.87,88.0,37.0,9.0,37.0,EA Sports,T
5320,Mirror's Edge Catalyst,PS4,2016.0,Platform,Electronic Arts,0.11,0.16,0.02,0.05,0.35,69.0,50.0,4.7,578.0,EA DICE,T
8492,Mirror's Edge Catalyst,XOne,2016.0,Platform,Electronic Arts,0.09,0.06,0.00,0.01,0.16,72.0,46.0,4.4,266.0,EA DICE,T
12369,Mirror's Edge Catalyst,PC,2016.0,Platform,Electronic Arts,0.01,0.05,0.00,0.00,0.06,74.0,24.0,4.4,606.0,EA DICE,T
2651,Need for Speed: Porsche Unleashed,PS,1999.0,Racing,Electronic Arts,0.43,0.29,0.00,0.05,0.77,78.0,13.0,8.5,24.0,Eden Studios,E
5234,Harry Potter and the Chamber of Secrets,XB,2002.0,Action,Electronic Arts,0.27,0.08,0.00,0.01,0.36,77.0,13.0,7.2,12.0,Eurocom Entertainment Software,E


In [65]:
game.iloc[suggest_me_games('need for speed', 30, remove_sequel=False),:]

your selection:
Name               Need for Speed: Shift
Platform                             PS3
Year_of_Release                     2009
Genre                             Racing
Publisher                Electronic Arts
NA_Sales                            0.69
EU_Sales                             1.4
JP_Sales                            0.04
Other_Sales                         0.52
Global_Sales                        2.65
Critic_Score                          84
Critic_Count                          55
User_Score                           6.9
User_Count                           113
Developer           Slightly Mad Studios
Rating                                 E
Name: 575, dtype: object
time taken to suggest: 0.1845388412475586 seconds


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
12682,Need for Speed: Shift,PC,2009.0,Racing,Electronic Arts,0.00,0.05,0.00,0.01,0.06,83.0,22.0,5.8,387.0,Slightly Mad Studios,E
863,Need for Speed Rivals,PS4,2013.0,Racing,Electronic Arts,0.73,0.92,0.03,0.30,1.98,80.0,53.0,6.2,667.0,Ghost Games,E10+
1549,The Sims 2: Nightlife,PC,2005.0,Simulation,Electronic Arts,1.22,0.05,0.00,0.00,1.27,76.0,31.0,6.6,71.0,Maxis,T
2712,Harry Potter and the Half-Blood Prince,Wii,2009.0,Action,Electronic Arts,0.28,0.39,0.00,0.08,0.75,60.0,35.0,6.0,8.0,EA Bright Light,E10+
9665,Charm Girls Club: My Fashion Mall,DS,NaN,Simulation,Unknown,0.12,0.00,0.00,0.01,0.12,0.0,0.0,0.0,0.0,Electronic Arts,E
847,Battlefield 4,XOne,2013.0,Shooter,Electronic Arts,1.25,0.58,0.00,0.19,2.02,81.0,5.0,6.6,480.0,EA DICE,M
8482,F1 Championship Season 2000,PS2,2000.0,Racing,Electronic Arts,0.06,0.05,0.03,0.02,0.16,71.0,11.0,0.0,0.0,EA Sports,E
849,Need for Speed: The Run,PS3,2011.0,Action,Electronic Arts,0.58,1.04,0.03,0.36,2.01,64.0,48.0,6.4,182.0,Black Box,T
851,Dragon Age: Inquisition,PS4,2014.0,Role-Playing,Electronic Arts,0.73,0.88,0.08,0.32,2.01,89.0,43.0,7.4,1984.0,BioWare,M
9668,Need for Speed: ProStreet,DS,2007.0,Racing,Electronic Arts,0.11,0.01,0.00,0.01,0.12,74.0,8.0,7.4,10.0,Exient Entertainment,E
